# Retargeting results report

Here is the code that is used in a real project to estimate the results of retargeting campaign.

So, we have some customers that are gone, but we try to return them back.

To calculate the results, we need 2 tables:

- All the orders for some period of time with customers identifier;
- Identifiers of customers that we tried to return.

In [1]:
from lemuras import Table

### Helpful functions

In [2]:
# Normalizes phone number
def filterPhone(x):
    try:
        return str(int(x))[-10:]
    except:
        return ''


# Returns index of phone numbes column
def findTels(people):
    for col in people.columns:
        if ('tel' in col) or ('phone' in col):
            return people.column_indices[col]
    return None


# Returns index of email column
def findMail(people):
    for col in people.columns:
        if 'email' in col:
            return people.column_indices[col]
    return None

## Load and preprocess orders table

The table contains all the orders of all the customers for some period of time.

In [3]:
orders = Table.fromCsv('data/orders.csv', False)

if not ('tel' in orders.columns and 'client_id' in orders.columns and 'total' in orders.columns):
    print('Wrong orders table!')

orders.apply('tel', filterPhone)

orders

'client_id','dt','total','bonus','tel','email'
'npOCzzDgwf3RRrSw','2018-02-08',540,0,'7355155113','afgs-71@mail.ru'
'fvJurzvTg8vaRp3J','2018-02-08',1000,0,'7037259419','cctgrafp4cc3777@yandex.ru'
'4fr4na4v8vOw33s7','2018-02-08',1010,0,'7595533171','rcftgfp4cc@list.ru'
...,...,...,...,...,...


## Load and preprocess people table

The table contains identifiers of the customers that we tried to return.

In [4]:
people = Table.fromCsv('data/people.csv', False)

# Let's imagine that the table is loaded by some user
# and we don't know the real structure.
# Thus, we need to find the column.
people_tel_ind = findTels(people)
if people_tel_ind is None:
    print('Wrong people table!')

people.apply(people_tel_ind, lambda x: filterPhone(x))

people

'tel'
'7752253123'
'7755253330'
'7753253525'
...


# Select orders by our people

In [5]:
people_tel = people.columnByIndex(people_tel_ind)
orders_tel = orders.columnByName('tel')

checker = orders_tel.isin(people_tel)
print('Found by phone: ' + str(checker.count()))

people_mail_ind = findMail(people)
if people_mail_ind is not None:
    people_mail = people.columnByIndex(people_mail_ind)
    orders_mail = orders.columnByName('email')
    c_mail = (orders_mail.isin(people_mail) & (orders_mail.lengths() > 5))
    print('Found by mail: ' + str(c_mail.count()))
    checker = checker | c_mail

united = orders.loc(checker)

united.toCsv('data/united.csv')
united

Found by phone: 100


'client_id','dt','total','bonus','tel','email'
'vTzQyRgpAz1Dzzrd','2018-02-08',880,0,'7011255934','87788433718@mail.ru'
'8QCpTluUfnC8uAsp','2018-02-08',400,110,'7475053415',''
'zpzqzAzRXcnpOaXQ','2018-02-08',600,0,'7573050559','ngtucrpzccffc@rambler.ru'
...,...,...,...,...,...


# Returned customers

In [6]:
byclient = united.groupby(['tel', 'email', 'client_id'])
byclient = byclient.agg({'total': {'TotalSum': 'sum', 'TotalAvg': 'mean', 'Orders': 'count'}})

byclient.toCsv('data/cli.csv')
byclient

'tel','email','client_id','TotalSum','TotalAvg','Orders'
'7011255934','87788433718@mail.ru','vTzQyRgpAz1Dzzrd',1270,635.0,2
'7475053415','','8QCpTluUfnC8uAsp',400,400.0,1
'7573050559','ngtucrpzccffc@rambler.ru','zpzqzAzRXcnpOaXQ',600,600.0,1
...,...,...,...,...,...


# Summary results

In [7]:
res = united.groupby()
res = res.agg({'total': {'TotalSum': 'sum', 'TotalAvg': 'mean'}})

ful = res.cell('TotalSum')
avg = res.cell('TotalAvg')

print('- Results:')
print('People in group: ' + str(people.rowcnt))
print('People returned: ' + str(byclient.rowcnt))
print('Retention: ' + str(int(1000.0 * byclient.rowcnt / people.rowcnt) / 10.0) + '%')
print('Orders number' + str(united.rowcnt))
if (byclient.rowcnt > 0):
    print('Orders per person: ' + str(int(100.0 * united.rowcnt / byclient.rowcnt) / 100.0))
print('Average check: ' + str(int(avg)))
print('Total sum: ' + str(int(ful)))

- Results:
People in group: 4295
People returned: 82
Retention: 1.9%
Orders number100
Orders per person: 1.21
Average check: 1206
Total sum: 120613
